In [88]:
import copy
from collections import deque



In [1]:
def read_a_graph(file_name):
    %pwd
    %cd /Users/AM/Documents/_Courses/Algorithms/Misc
    # load a comma-delimited text file into an np matrix
    resultList = {}
    f = open(file_name, 'r')
    for line in f:
        aa = line.rstrip('\n')  # "1.0 \t 2.0 \t 3.0"
        sVals = aa.split('\t')
        sVals_s = sVals[:-1] # remove the last ''
        intVals = list(map(np.int, sVals_s))  # [1.0, 2.0, 3.0]
        resultList[intVals[0]] = intVals[1:]
    
    f.close()
    out = resultList
    return out#np.asarray(resultList, dtype=np.int)  

In [147]:
# Programming assignment 2.1 Programming assignment. MIN CUT
class Graph(object):

    def __init__(self):
        self.al = {} # adjacency list for the graph (self)
        self.al_r = {} # adjacency list for the reversed graph (self)
        self.max_i = 0 # max index of the graph nodes (or verteces)
        self.explr = [] # exploration flags for DFF; 
        self.ft_r = [] # b - finishing time forvard
                                        # c - finishing time reversed

    
    def read_direct(self, file_name):
        %pwd
        %cd /Users/AM/Documents/_Courses/Algorithms/Misc
        # load a comma-delimited text file into an np matrix
        resultList = {}
        f = open(file_name, 'r')
        max_i = self.max_i
        for line in f:
            line_s = line.rstrip('\n')  # "1.0 2.0 3.0"    
            sVals = line_s.split(' ')
            sVals_s = sVals[:-1] # remove the last ''
            intVals = list(map(np.int, sVals_s))  # [1.0, 2.0, 3.0]
            if intVals[0] > max_i: max_i = intVals[0] # looking for a vertex with the max index
            if intVals[1] > max_i: max_i = intVals[1]

            if intVals[0] in resultList.keys(): 
                resultList[intVals[0]].append(intVals[1])
            else:
                resultList[intVals[0]] = [intVals[1]]
        
        f.close()
        self.al = resultList
        self.max_i = max_i
        self.explr = [False for x in range(self.max_i + 1)]
        self.ft = [0 for x in range(self.max_i + 1)]

    
    def read_reverse(self, file_name):
        %pwd
        %cd /Users/AM/Documents/_Courses/Algorithms/Misc
        # load a comma-delimited text file into an np matrix
        resultList = {}
        f = open(file_name, 'r')
        max_i = self.max_i ##!!!##to do - avoid the overwrite of max_i from read_direct
                    
        for line in f:
            line_s = line.rstrip('\n')  # "1.0 2.0 3.0"    
            sVals = line_s.split(' ')
            sVals_s = sVals[:-1] # remove the last ''
            intVals = list(map(np.int, sVals_s))  # [1.0, 2.0, 3.0]
            if intVals[0] > max_i: max_i = intVals[0] # looking for a vertex with the max index
            if intVals[1] > max_i: max_i = intVals[1]
            
            if intVals[1] in resultList.keys(): 
                resultList[intVals[1]].append(intVals[0])
            else:
                resultList[intVals[1]] = [intVals[0]]
        
        f.close()
        self.al_r = resultList
        self.max_i = max_i
        self.explr = [False for x in range(self.max_i + 1)]
        self.ft_r = [0 for x in range(self.max_i + 1)]
     
    def dfs_rev(self):
        self.explr = [False for x in range(self.max_i + 1)] # reset explored flags
        q_dfs = deque([list(gr.al_r.keys())[0]]) # add to a dfs queue the first vertex 
        fin_time = 0
        
        while q_dfs:
            i = q_dfs.pop() # take a next node to explore
            self.explr[i] = True # mark as explored the node i
            self.al_r[i].sort() # make a sorted and reversed list of all the nodes 
            self.al_r[i].reverse()   ### !!! ### it might be empty
            a = self.al_r[i]    
            
            new_vertex_discovered = False
            for ver in a:
                if self.explr[ver] == False: # if we find a new vertex
                    q_dfs.append(ver)
                    self.explr[ver] = True 
                    new_vertex_discovered = True
            if new_vertex_discovered == False:
                fin_time += 1
                self.ft_r[i] = fin_time

        
    
#    def scc
    
    
    
    
    def connect_vertices(self, v1,v2):
        '''
        Connect vertices v1 and v2 and removes edges between v1 and v2
        Parallel edges from the newly formed verix (v1+v2) are not removed
        '''
        remaining_vertex = min(v1,v2)
        disappearing_vertex = max(v1,v2)
        self.al[remaining_vertex] += self.al[disappearing_vertex]
            
        # delete the second vertex    
        del self.al[disappearing_vertex] 
        # replace disappearing vertex for remaining in all the verteces
        for i in list(self.al):
            while self.al[i].count(disappearing_vertex) !=0:
                self.al[i].remove(disappearing_vertex)
                self.al[i].append(remaining_vertex)
        # remove edges from remaining to remaining vertex
        while self.al[remaining_vertex].count(remaining_vertex) !=0:
            self.al[remaining_vertex].remove(remaining_vertex)  
    
    def find_two_rand_vert(self, seed = 111):
        '''choses two random verteces from the remaining in self.al'''
        #ver1 = rnd.choice(list(self.al) ,1 , replace = False)
        ver1 = rnd.choice(list(self.al.keys()))
        ver2 = rnd.choice(list(self.al[ver1]))
        return ver1, ver2
    
    def find_a_cut(self):
        '''merge v1 and v2 verteces and removes the one with higher index
        and returns number of remaining vert'''
        while len(list(self.al)) > 2:
            ver = self.find_two_rand_vert()
            self.connect_vertices(ver[0], ver[1])
        return len(self.al[list(self.al)[0]])    
           
    def find_min_cut(self):
        '''by randomly merging by 2 verteces returns number of edges between the final two'''
        min_cut_len = 1000 #len(list(self.al))
        al_master = copy.deepcopy(self.al)
        for i in range(30):
            self.al = copy.deepcopy(al_master)
            x = self.find_a_cut()
            print('current iteration_',i, end = '\r')
            if min_cut_len > x: 
                min_cut_len = x
                al_min_cut = copy.deepcopy(self.al)
                print('\r min_cut_len found after iteration_',i, 'is:', x, '\n')

        self.al = copy.deepcopy(al_master)
        return min_cut_len, al_min_cut
    
    def test(self):
        g = {1:[2,4,5], 2:[1,3], 3:[2,4,5], 4:[1,3,5], 5:[1,3,4]}
        self.al = g
        print('before connection\n', self.al)
        self.connect_vertices(5,2)
        print('after 1st connection\n', self.al)
        self.connect_vertices(3,4)
        print('after 2st connection\n', self.al)

    def test2(self):
        g = {1:[2,4,5], 2:[1,3], 3:[2,4,5], 4:[1,3,5], 5:[1,3,4]}
        self.al = g
        print('before the test 2 \n', self.al)
        l = self.find_a_cut()
        print('test results:\n', self.al, 'length of the found cut', l)
    

In [148]:
gr = Graph()
gr.read_direct('text4.txt')

/Users/AM/Documents/_Courses/Algorithms/misc


In [149]:
gr.max_i

875714

In [150]:
gr.read_reverse('text4.txt')


/Users/AM/Documents/_Courses/Algorithms/misc


In [151]:
gr.max_i

875714

In [152]:
gr.dfs_rev()

KeyError: 774055

In [139]:
a = [3,5,4,1,7]
a.sort()
a.reverse()
for i in a:
    print(i)

7
5
4
3
1
